In [1]:
from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

import numpy as np
import pandas as pd
import image_utils as iu
import tensorflow as tf
import os
from PIL import Image, ImageEnhance
from PIL import ImageFilter
from keras.models import load_model
from shutil import copy2
import matplotlib.pyplot as plt
import numpy_utils as nu

In [6]:
data_path = 'data'
segmentation_path = os.path.join(data_path, 'segmentation')
# Saved models
saved_model_path = os.path.join(segmentation_path, 'saved-models')
# Predictions path.
prediction_path = os.path.join(segmentation_path, 'predictions')
#
original_path = os.path.join(segmentation_path, 'original')
real_data_path = os.path.join(original_path, 'real')
mask_data_path = os.path.join(original_path, 'mask')


In [9]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

# Predicts the mask of an image.
def predict_mask(unet, np_img, input_size):
    # split image
    image_width, image_height = input_size
    np_img = nu.resize_np_image(np_img, (image_height, image_width))
    np_img = np_img / 13029
    pred = unet.predict(np.array([np_img]))
    return np.asarray(create_mask(pred))

In [21]:
def split_image(image, row_count, col_count, new_part_size):
    # parts = []
    width, height = image.size
    left = 0
    top = 0
    right = width / col_count
    bottom = height / row_count
    rows = []
    for r in range(row_count):
        row = []
        top = int(r * (height / row_count))
        bottom = int(top + (height / row_count))
        for c in range(col_count):
            left = int(c * (width / col_count))
            right = int(left + (width / col_count))
            part = image.crop((left, top, right, bottom))
            # part = part.resize(new_part_size)
            # parts.append(part)
            row.append(part)
        rows.append(row)
    return rows


def crop_geoTiff(geoTiff, left, top, right, bottom, dtype='uint16'):
    width = abs(right - left)
    height = abs(top - bottom)
    bands = []
    for x in range(geoTiff.RasterCount):
        band = geoTiff.GetRasterBand(x + 1).ReadAsArray(left, top,
                                                        int(width), int(height))
        bands.append(band)
    output = np.zeros(
        (int(height), int(width), geoTiff.RasterCount), dtype)
    for x in range(len(bands)):
        output[..., x] = bands[x]
    return output

def split_geoTiff_image_into_rows_and_columns(geoTiff, row_count, col_count):
    # parts = []
    width, height = geoTiff.RasterXSize, geoTiff.RasterYSize
    left = 0
    top = 0
    right = width / col_count
    bottom = height / row_count
    rows = []
    # bands = read_geoTiff_bands(geoTiff)
    for r in range(row_count):
        row = []
        top = int(r * (height / row_count))
        bottom = int(top + (height / row_count))
        for c in range(col_count):
            left = int(c * (width / col_count))
            right = int(left + (width / col_count))
            part = crop_geoTiff(geoTiff, left, top, right, bottom)
            row.append(part)
        rows.append(row)
    return rows


def split_image_into_rows(geoTiff, max_divided_image_size):
    max_d_width, max_d_height = max_divided_image_size
    image_width, image_height = geoTiff.RasterXSize, geoTiff.RasterYSize
    row_count, col_count = int(
        image_height / max_d_height), int(image_width / max_d_width)
    rows = split_geoTiff_image_into_rows_and_columns(geoTiff, row_count, col_count)
    return rows


# empty_img = np.zeros([512, 1024], dtype=np.uint8)
# empty_img.fill(0)

def combine_predictions(rows, color_space='L'):
    part_height, part_width = rows[0][0].shape
    full_image_width, full_image_height = (
        len(rows[0]) * part_width, len(rows) * part_height)
    empty_img = np.zeros([full_image_height, full_image_width], dtype=np.uint8)
    empty_img = Image.fromarray(empty_img).convert(color_space)
    for x in range(len(rows)):
        row = rows[x]
        for i in range(len(row)):
            image_part = row[i]
            empty_img.paste(image_part, (i * part_width, x * part_height))
    return empty_img

def predict_image_parts(unet, rows,image_size):
    new_rows = []
    for row in rows:
        new_row = []
        for np_image in row:
            print(np_image.shape)
            mask = predict_mask(unet, np_image, image_size)  
            new_row.append(mask)
        new_rows.append(new_row)
    return new_rows


In [5]:
mc_unet = load_model(os.path.join(
    saved_model_path, 'multi_class_unet_model_2023-01-04 09-50-02.700072.h5'))

2023-01-04 11:11:54.039106: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
real_im = read_geotiff_as_rgb_image(
    'data/segmentation/PLANET IMAGES FROM WEBSITE/2015/Images_Dec15/L15-1009E-1056N.tif')
im = real_im[0]
# rows = split_image_into_rows(real_image, (512, 426))
image_size = (320, 160)
rows = split_image_into_rows(im, image_size)
rows = predict_image_parts(unet, None, rows,  0.9, image_size)
new_im = combine_predictions(rows)

In [7]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 392861216
real_image = gdal.Open(os.path.join(
    real_data_path, 'Sentinel_2_2017_cropped.tif'))
mask_image = iu.read_image(os.path.join(
    mask_data_path, 'Final_RLCM_Ghana_2017.png'))

In [22]:
image_size = (160, 160)
rows = split_image_into_rows(real_image, image_size)
rows = predict_image_parts(mc_unet, rows, image_size)
land_map = combine_predictions(rows)

(160, 160, 6)


2023-01-04 11:31:38.915478: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [20]:
rows[0][0].shape

(160, 6)